In [2]:
#loading the necessary libraries
import pandas as pd 
from sklearn.utils import resample
from textblob import TextBlob
import codecs
import re
import spacy
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
import sklearn
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils.validation import check_array
#from sklearn.utils import check_array
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
import emoji
from textblob import TextBlob
import codecs
#import unidecode
import re
import spacy
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
#loading previously saved cleaned dataframe
class_df=pd.read_excel('dataframe_clean.xlsx')
class_df.shape

(194076, 14)

In [294]:
#loading the original dataset
dme = pd.read_excel("google__reviews_tunein_radio.xlsx", index_col=0)
dme.shape

(194229, 8)

In [296]:
#removing observations that have missing values in the variable content
dme.dropna(subset = ["content"], inplace=True)
dme.shape

(193277, 8)

In [297]:
#replacing the scores with the sentiment
dme['score'] = dme['score'].replace(5,'pos')
dme['score']= dme['score'].replace(4,'pos')
dme['score'] = dme['score'].replace(3,'neu')
dme['score'] = dme['score'].replace(2,'neg')
dme['score'] = dme['score'].replace(1,'neg')

In [298]:
dme.head(80)

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOEVAaqxNXyjz1hB3k7bEho_AxkJ7SrFwRwyyis...,I liike it,pos,0,25.9,2021/03/16,NaN,NaN
1,gp:AOqpTOFFp_buXI33ICX36w-olVNV5lFmecsHLkRzonr...,Good,pos,0,26.2.2,2021/03/16,NaN,NaN
2,gp:AOqpTOF8LXTtGD_LP6mw6LtOa3_sPBZA_9X5bDBpON2...,Wonder selection of music from all around the ...,pos,0,26.2.2,2021/03/16,NaN,NaN
3,gp:AOqpTOFsr7yM-5jNNc-SfkPgyA1SN-ErGk41mCpH3mD...,"I'm using TuneIn Radio since 2011 until now,th...",pos,0,26.2.2,2021/03/16,NaN,NaN
4,gp:AOqpTOH3n9ClNv33CGS_H2b9TEzNRkUKLI3EHsV6Z00...,App is Good but Reason Why is The Amazon App h...,neg,0,NaN,2021/03/16,NaN,NaN
...,...,...,...,...,...,...,...,...
75,gp:AOqpTOE57CUjWo-SZwyEmOYo-cpj-NyBDuxnL-VKPWL...,Absolute rubbish. A big con,neg,0,23.8.2,2021/03/13,Thanks for writing to us. We’re constantly wor...,2021/03/15
76,gp:AOqpTOEAdp_yY1pYXPmegDpb1bpodud73zGYcFKJHRf...,Excellent radio for oldies. TOP SHELF,pos,0,26.2.2,2021/03/13,NaN,NaN
77,gp:AOqpTOGwYOFDBdJvn8NKsdUq8_N1PQgz5INMPXck1hT...,In the UK the app WILL NOT play thousands of r...,neg,0,21.4.1,2021/03/13,Thank you for writing to us. Unfortunately due...,2021/03/15
78,gp:AOqpTOGWScG1iM-mIUKQrCdHcFLwYJPzFgJzPOrcokY...,Awesome Radio!!!,pos,0,26.1,2021/03/13,NaN,NaN


In [299]:
#leaving this balanced set of data (15 observations) out for testing with VADER and all of the models I am going to create
sent_analysis=dme[dme.index.isin([0,1,2,3,4,5,8,14,16,28,29,58,89,103,107])]
sent_analysis

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOEVAaqxNXyjz1hB3k7bEho_AxkJ7SrFwRwyyis...,I liike it,pos,0,25.9,2021/03/16,NaN,NaN
1,gp:AOqpTOFFp_buXI33ICX36w-olVNV5lFmecsHLkRzonr...,Good,pos,0,26.2.2,2021/03/16,NaN,NaN
2,gp:AOqpTOF8LXTtGD_LP6mw6LtOa3_sPBZA_9X5bDBpON2...,Wonder selection of music from all around the ...,pos,0,26.2.2,2021/03/16,NaN,NaN
3,gp:AOqpTOFsr7yM-5jNNc-SfkPgyA1SN-ErGk41mCpH3mD...,"I'm using TuneIn Radio since 2011 until now,th...",pos,0,26.2.2,2021/03/16,NaN,NaN
4,gp:AOqpTOH3n9ClNv33CGS_H2b9TEzNRkUKLI3EHsV6Z00...,App is Good but Reason Why is The Amazon App h...,neg,0,NaN,2021/03/16,NaN,NaN
5,gp:AOqpTOEUe27aJj8sNGxHIJZ3x_IxvQvW9qbsGH-4QuM...,Good,pos,0,26.2.2,2021/03/16,NaN,NaN
8,gp:AOqpTOHCknZY3z0h1LzbqNOJUw09folVEE9Uncaiu-d...,Does not work with some of my favorite stations,neg,0,NaN,2021/03/16,NaN,NaN
14,gp:AOqpTOETATayONoers4A5s6hKIfAYlPlC7ymz-twNPR...,"DO NOT INSTALL, Sold my info, I get daily call...",neg,0,25.9,2021/03/15,Thanks for writing to us. We do not sell users...,2021/03/15
16,gp:AOqpTOGey7pRRVoMnpemXVbJ0GpZ1v1DraKtyEt-dTB...,Trying,neg,0,NaN,2021/03/15,Thanks for writing to us. If you need assistan...,2021/03/15
28,gp:AOqpTOH5vKNQ-AQiZ0v8Ta-xVap_bjlT3VIio3DMRkR...,Nice lots of options. Buffers a lot regardless...,neu,0,26.2.2,2021/03/15,Thank you for writing to us. That shouldn't be...,2021/03/15


In [300]:
sent_analysis.to_excel('vader.xlsx')

In [28]:
list_dme=[0,1,2,3,4,5,8,14,16,28,29,58,89,103,107]


In [32]:
#excluding the saved 15 observations from the data for modleing
indexes_to_keep = set(range(dme.shape[0])) - set(list_dme)
df_sliced = dme.take(list(indexes_to_keep))
df_sliced.head()

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
6,gp:AOqpTOF3QCVRdpE5LPG4_w4kxMGZM3QM6S-_q9bA1nI...,Easy to use.,pos,0,26.2.2,2021/03/16,NaN,NaN
7,gp:AOqpTOGpCugDJB9oGVjeelpn5Ts6HNRypKXNfGgUdgk...,Awsom,pos,0,26.2.2,2021/03/16,NaN,NaN
9,gp:AOqpTOE0JfIVtjBUaWdq4IPtCWgXQugpgQ0iB70HmB6...,I need to download tunein radio,pos,0,NaN,2021/03/16,NaN,NaN
10,gp:AOqpTOFQQxYFdlL7mfuYvYYBW2hrKhDs5mSzaf4PQ1p...,Splendiferous,pos,0,25.7.3,2021/03/16,NaN,NaN
11,gp:AOqpTOH5XvsOf3UVHpr0Zx7d7spXV6a9616sNrumWJZ...,Just a good app easy to use lots of good radio...,pos,0,26.2.2,2021/03/16,NaN,NaN


In [50]:
#saving it to an xlsx file
df_sliced.to_excel('sliced.xlsx')

In [33]:
#looking at the imbalance of the classes of the cleaned dataframe
df_sliced.shape

(193262, 8)

In [36]:
df_sliced_neg = df_sliced[df_sliced['score']=='neg']
df_sliced_neg.shape

(29879, 8)

In [37]:
df_sliced_neu = df_sliced[df_sliced['score']=='neu']
df_sliced_neu.shape

(9990, 8)

In [39]:
df_sliced_pos=df_sliced[df_sliced['score']=='pos']
df_sliced_pos.shape

(153240, 8)

In [40]:
from sklearn.utils import resample

In [41]:
#downsampling 
df_sliced_neg_downsampled = resample(df_sliced_neg, replace=False, n_samples = 9990,random_state=123)

In [42]:
#downsampling
df_sliced_pos_downsampled = resample(df_sliced_pos, replace=False, n_samples = 9990,random_state=123)

In [44]:
#creating a balanced downsampled dataset
df_downsampled = pd.concat([df_sliced_pos_downsampled,df_sliced_neg_downsampled,df_sliced_neu])
df_downsampled.score.value_counts()

neu    9990
neg    9990
pos    9990
Name: score, dtype: int64

In [45]:
#saving it to a dataframe
df_downsampled.to_excel('downsampled.xlsx')

In [51]:
#removing emojis
import emoji
df_sliced_cleaned = df_sliced
df_sliced_cleaned['content'] = df_sliced['content'].astype(str).apply(lambda x: emoji.demojize(x, delimiters=("", "")))
df_sliced_cleaned['content'] =df_sliced_cleaned['content'].replace(to_replace=r"’",value="'",regex=True)

In [71]:
#converting to lowercase
df_sliced_cleaned['content'] =df_sliced_cleaned['content'].str.lower()

In [31]:
#a list of contractions and the full form
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not","Doesn't":"Does not",
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 

In [73]:
#substituting the contractions with the full form
df_sliced_cleaned['content'] =df_sliced_cleaned['content'].apply(lambda item: ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in item.split(" ")]))


In [74]:
df_sliced_cleaned.head(50)

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
6,gp:AOqpTOF3QCVRdpE5LPG4_w4kxMGZM3QM6S-_q9bA1nI...,easy to use.,pos,0,26.2.2,2021/03/16,NaN,NaN
7,gp:AOqpTOGpCugDJB9oGVjeelpn5Ts6HNRypKXNfGgUdgk...,awsom,pos,0,26.2.2,2021/03/16,NaN,NaN
9,gp:AOqpTOE0JfIVtjBUaWdq4IPtCWgXQugpgQ0iB70HmB6...,i need to download tunein radio,pos,0,NaN,2021/03/16,NaN,NaN
10,gp:AOqpTOFQQxYFdlL7mfuYvYYBW2hrKhDs5mSzaf4PQ1p...,splendiferous,pos,0,25.7.3,2021/03/16,NaN,NaN
11,gp:AOqpTOH5XvsOf3UVHpr0Zx7d7spXV6a9616sNrumWJZ...,just a good app easy to use lots of good radio...,pos,0,26.2.2,2021/03/16,NaN,NaN
12,gp:AOqpTOE6pI6KEb4if2pPovllcG-NA41EsIr-BBEiTpq...,works as expected.,pos,38,26.2.2,2021/03/15,NaN,NaN
13,gp:AOqpTOHLAe1C0aztXdANYzZllOnnHZ05QBQLo2rAo4f...,insuperables!,pos,0,26.2.2,2021/03/15,NaN,NaN
15,gp:AOqpTOG2gbnIZ-0hfRfm1XNWaq4tVrpSs1dVpgpV-lX...,love it,pos,0,26.1,2021/03/15,NaN,NaN
17,gp:AOqpTOEFW3yq5AlwZq9ro2qiXXwqg2qjYugdIIZfzhY...,all good,pos,0,26.2.2,2021/03/15,We appreciate you saying so!\n,2021/03/15
18,gp:AOqpTOH1ivrx9btJD9DExMzxUic6vLMevVtVEUgxiPW...,cool,pos,0,NaN,2021/03/15,We aim to please!\n,2021/03/15


In [75]:
#removing underscore that resulted in the emoji conversion
df_sliced_cleaned['content'] =df_sliced_cleaned['content'].replace(to_replace=r"_",value=" ",regex=True)
df_sliced_cleaned.head(20)

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
6,gp:AOqpTOF3QCVRdpE5LPG4_w4kxMGZM3QM6S-_q9bA1nI...,easy to use.,pos,0,26.2.2,2021/03/16,NaN,NaN
7,gp:AOqpTOGpCugDJB9oGVjeelpn5Ts6HNRypKXNfGgUdgk...,awsom,pos,0,26.2.2,2021/03/16,NaN,NaN
9,gp:AOqpTOE0JfIVtjBUaWdq4IPtCWgXQugpgQ0iB70HmB6...,i need to download tunein radio,pos,0,NaN,2021/03/16,NaN,NaN
10,gp:AOqpTOFQQxYFdlL7mfuYvYYBW2hrKhDs5mSzaf4PQ1p...,splendiferous,pos,0,25.7.3,2021/03/16,NaN,NaN
11,gp:AOqpTOH5XvsOf3UVHpr0Zx7d7spXV6a9616sNrumWJZ...,just a good app easy to use lots of good radio...,pos,0,26.2.2,2021/03/16,NaN,NaN
12,gp:AOqpTOE6pI6KEb4if2pPovllcG-NA41EsIr-BBEiTpq...,works as expected.,pos,38,26.2.2,2021/03/15,NaN,NaN
13,gp:AOqpTOHLAe1C0aztXdANYzZllOnnHZ05QBQLo2rAo4f...,insuperables!,pos,0,26.2.2,2021/03/15,NaN,NaN
15,gp:AOqpTOG2gbnIZ-0hfRfm1XNWaq4tVrpSs1dVpgpV-lX...,love it,pos,0,26.1,2021/03/15,NaN,NaN
17,gp:AOqpTOEFW3yq5AlwZq9ro2qiXXwqg2qjYugdIIZfzhY...,all good,pos,0,26.2.2,2021/03/15,We appreciate you saying so!\n,2021/03/15
18,gp:AOqpTOH1ivrx9btJD9DExMzxUic6vLMevVtVEUgxiPW...,cool,pos,0,NaN,2021/03/15,We aim to please!\n,2021/03/15


In [77]:
#spellchecking
df_sliced_cleaned['content'] =df_sliced_cleaned['content'].apply(lambda txt: ''.join(TextBlob(txt).correct()))

In [78]:
df_sliced_cleaned['content'].head(20)

6                                          easy to use.
7                                                  also
9                       i need to download tushin radio
10                                        splendiferous
11    just a good pp easy to use lots of good radio ...
12                                   works as expected.
13                                         insuperable!
15                                              love it
17                                             all good
18                                                 cool
19                                     super strong and
20    great little pp for music, news, and sports! l...
21    pretty decent pp, i get most music i want to h...
22                           tushin radio is my several
23                                                great
24                                           every nice
25             very good smiling face with smiling eyes
26                                    big things

In [79]:
#saving the clean dataset
df_sliced_cleaned.to_excel('spellchecked_oversampled.xlsx')

In [82]:
nlp = spacy.load("en_core_web_sm")

In [83]:
#cleaning the data
def spacy_cleaner(text):
    parsed = nlp(text)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [84]:
df_sliced_cleaned['clean_text'] = [spacy_cleaner(t) for t in df_sliced_cleaned.content]
df_sliced_cleaned.head()

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,clean_text
6,gp:AOqpTOF3QCVRdpE5LPG4_w4kxMGZM3QM6S-_q9bA1nI...,easy to use.,pos,0,26.2.2,2021/03/16,NaN,NaN,easy to use
7,gp:AOqpTOGpCugDJB9oGVjeelpn5Ts6HNRypKXNfGgUdgk...,also,pos,0,26.2.2,2021/03/16,NaN,NaN,also
9,gp:AOqpTOE0JfIVtjBUaWdq4IPtCWgXQugpgQ0iB70HmB6...,i need to download tushin radio,pos,0,NaN,2021/03/16,NaN,NaN,need to download tushin radio
10,gp:AOqpTOFQQxYFdlL7mfuYvYYBW2hrKhDs5mSzaf4PQ1p...,splendiferous,pos,0,25.7.3,2021/03/16,NaN,NaN,splendiferous
11,gp:AOqpTOH5XvsOf3UVHpr0Zx7d7spXV6a9616sNrumWJZ...,just a good pp easy to use lots of good radio ...,pos,0,26.2.2,2021/03/16,NaN,NaN,just good pp easy to use lot of good radio sta...


In [98]:
#saving the data to an xlsx file
df_sliced_cleaned.to_excel('clean_func.xlsx')

In [4]:
df_sliced_cleaned = pd.read_excel('clean_func.xlsx')

In [5]:
df_sliced_cleaned['clean_text'] = df_sliced_cleaned['clean_text'].astype(str)
df_sliced_cleaned['score'] = df_sliced_cleaned['score'].astype(str)

In [6]:
#removing the zeros from the column score
df_sliced_cleaned['score'].unique()

array(['pos', 'neg', 'neu', '0'], dtype=object)

In [7]:
df_sliced_cleaned_new= df_sliced_cleaned[df_sliced_cleaned['score']!= '0']
df_sliced_cleaned_new['score'].unique()

array(['pos', 'neg', 'neu'], dtype=object)

In [8]:
#resetting the index
df_sliced_cleaned_new.reset_index(drop=True, inplace=True)

In [78]:
#creating a function that will cross validate a model and print out the results
tvec = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1, 3))
lr = LogisticRegression()


def lr_cv(splits, X, Y, pipeline, average_method):
    
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=777)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test],Y[test])
        
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        print('              negative    neutral     positive')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

In [80]:
#checking the performance of the original imbalanced data set for logistic regression model
lr_cv(5, df_sliced_cleaned_new.clean_text, df_sliced_cleaned_new.score, original_pipeline,'macro')

              negative    neutral     positive
precision: [0.76249592 0.36094675 0.92730552]
recall:    [0.7811245  0.06106106 0.97311407]
f1 score:  [0.7716978  0.10445205 0.9496577 ]
--------------------------------------------------
              negative    neutral     positive
precision: [0.76741536 0.40895522 0.92922254]
recall:    [0.78898929 0.06856857 0.97454973]
f1 score:  [0.77805281 0.11744535 0.95134653]
--------------------------------------------------
              negative    neutral     positive
precision: [0.76490996 0.37142857 0.92707881]
recall:    [0.77476573 0.05855856 0.9756591 ]
f1 score:  [0.7698063  0.10116732 0.95074878]
--------------------------------------------------
              negative    neutral     positive
precision: [0.76269531 0.39057239 0.92806314]
recall:    [0.78413655 0.05805806 0.97448447]
f1 score:  [0.77326733 0.10108932 0.95070748]
--------------------------------------------------
              negative    neutral     positive
precision

In [76]:
ROS_pipeline = make_pipeline(tvec, RandomOverSampler(random_state=777),lr)

In [79]:
#checking the performance for the randomly oversampled data
lr_cv(5, df_sliced_cleaned_new.clean_text, df_sliced_cleaned_new.score, ROS_pipeline, 'macro')

              negative    neutral     positive
precision: [0.70787377 0.21088435 0.96952889]
recall:    [0.77325971 0.41891892 0.88971548]
f1 score:  [0.73912348 0.28054299 0.92790907]
--------------------------------------------------
              negative    neutral     positive
precision: [0.71790152 0.19441748 0.96883822]
recall:    [0.78313253 0.4009009  0.8845928 ]
f1 score:  [0.74909964 0.26185028 0.92480087]
--------------------------------------------------
              negative    neutral     positive
precision: [0.70498026 0.19777306 0.96709595]
recall:    [0.7769411  0.37337337 0.89186896]
f1 score:  [0.7392135  0.25857886 0.92796035]
--------------------------------------------------
              negative    neutral     positive
precision: [0.70511841 0.17459643 0.96733531]
recall:    [0.77225569 0.41141141 0.86384103]
f1 score:  [0.73716157 0.24515359 0.91266353]
--------------------------------------------------
              negative    neutral     positive
precision

In [70]:
SMOTE_pipeline = make_pipeline(tvec, SMOTE(random_state=777),lr)

In [81]:
#checking the performance for oversampled data with SMOTE
lr_cv(5, df_sliced_cleaned_new.clean_text, df_sliced_cleaned_new.score, SMOTE_pipeline, 'macro')

              negative    neutral     positive
precision: [0.66961231 0.18380213 0.96324867]
recall:    [0.78614458 0.37937938 0.86374315]
f1 score:  [0.72321429 0.24763149 0.91078617]
--------------------------------------------------
              negative    neutral     positive
precision: [0.69327302 0.1828167  0.96303122]
recall:    [0.79501339 0.37487487 0.8695184 ]
f1 score:  [0.74066568 0.24577523 0.91388889]
--------------------------------------------------
              negative    neutral     positive
precision: [0.69544693 0.14447925 0.95984711]
recall:    [0.76422356 0.36936937 0.84393761]
f1 score:  [0.72821494 0.20771179 0.89816824]
--------------------------------------------------
              negative    neutral     positive
precision: [0.68854645 0.18387175 0.96105347]
recall:    [0.77761044 0.37887888 0.87036674]
f1 score:  [0.73037328 0.2475879  0.91346483]
--------------------------------------------------
              negative    neutral     positive
precision

Preprocessing the downsampled data


In [25]:
df_downsampled_clean=pd.read_excel('downsampled.xlsx')
df_downsampled_clean.shape

(29970, 9)

In [26]:
df_downsampled_clean['content'] = df_downsampled_clean['content'].astype(str).apply(lambda x: emoji.demojize(x, delimiters=("", "")))

In [27]:
df_downsampled_clean['content'] =df_downsampled_clean['content'].replace(to_replace=r"_",value=" ",regex=True)

In [30]:
df_downsampled_clean['content'] =df_downsampled_clean['content'].replace(to_replace=r"’",value="'",regex=True)
#converting to lowercase
df_downsampled_clean['content']=df_downsampled_clean['content'].str.lower()

In [32]:
df_downsampled_clean['content']=df_downsampled_clean['content'].apply(lambda item: ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in item.split(" ")]))

In [35]:
#spell checking
df_downsampled_clean['content'] =df_downsampled_clean['content'].apply(lambda txt: ''.join(TextBlob(txt).correct()))

In [36]:
nlp = spacy.load("en_core_web_sm")

In [37]:
#lemmatizing the data
def spacy_cleaner(text):
    parsed = nlp(text)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [38]:
# applying the function above
df_downsampled_clean['clean_text'] = [spacy_cleaner(t) for t in df_downsampled_clean.content]
df_downsampled_clean.head()

,Unnamed: 0,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,clean_text
0,44862,gp:AOqpTOGWDeuYh12nc_I5Am6IzI-NXptmNFszVhtE42k...,"good solid pp. as an expect, i find it invalua...",pos,0,20.8,2018/10/21,NaN,NaN,good solid pp as an expect find it invaluable
1,131409,gp:AOqpTOFL0gICjMwsClvlFSqig04yUIQ1K9-Q9u_w_n5...,i love this radio.,pos,0,12.5,2014/08/16,NaN,NaN,love this radio
2,27730,gp:AOqpTOG1v6g2j_Ovz1Ln14Bw3fGquuHarkkcAP9N7Af...,they have some awesome live events from differ...,pos,0,22.6,2019/08/18,NaN,NaN,they have some awesome live event from differe...
3,188417,lg:AOqpTOEMlPuswifHJvR3m0jrMsdonXPsIvuFg4VAmeY...,"the audit is not he, but that is what you get ...",pos,0,1.5,2010/12/27,NaN,NaN,the audit be not he but that be what you get w...
4,182088,lg:AOqpTOF7P6FiwVn-SC5qahDUTfZkbRdcX_rEX96kwde...,loved the previous version,pos,0,2.0,2011/05/05,NaN,NaN,love the previous version


In [39]:
df_downsampled_clean.to_excel('downsampled_clean_func.xlsx')

In [40]:
df_downsampled_clean['clean_text'] = df_downsampled_clean['clean_text'].astype(str)
df_downsampled_clean['score'] = df_downsampled_clean['score'].astype(str)

In [46]:
#using the original pipeline to test the performance of the undersampled data
lr_cv(5, df_downsampled_clean.clean_text, df_downsampled_clean.score, original_pipeline, 'macro')

              negative    neutral     positive
precision: [0.6938873  0.60690045 0.77251407]
recall:    [0.72722723 0.53703704 0.82432432]
f1 score:  [0.71016618 0.56983537 0.79757869]
--------------------------------------------------
              negative    neutral     positive
precision: [0.68413462 0.60926431 0.8003848 ]
recall:    [0.71221221 0.55955956 0.83283283]
f1 score:  [0.69789112 0.58335507 0.81628649]
--------------------------------------------------
              negative    neutral     positive
precision: [0.70562347 0.62247191 0.78192716]
recall:    [0.72222222 0.55455455 0.84884885]
f1 score:  [0.71382637 0.58655373 0.81401488]
--------------------------------------------------
              negative    neutral     positive
precision: [0.70048544 0.61080178 0.77595884]
recall:    [0.72222222 0.54904905 0.83033033]
f1 score:  [0.71118778 0.5782815  0.80222437]
--------------------------------------------------
              negative    neutral     positive
precision

The choice of the data set used for the analysis is the downsampled data.
The reason for it is the metrics of precision, recall and F1.
Even though the accuracy is lower, the classes are predicted more evenly than when using SMOTE or RandomOversampling technique. 